# Chapter 2
Working with Tezos RPC: networks, chains, blocks, operations, and context.

In this chapter we will explore main blockchain structures and how to query them using PyTezos.  
Tezos node software is built in a [modular way](https://tezos.gitlab.io/whitedoc/the_big_picture.html), there is a generic **Shell** which is responsible for basic block validation, p2p, storage, and **Protocol** that contains main logic. Thanks to the [self-amendment](https://learn.tqtezos.com/files/self-amendment.html#introduction) feature this protocol code can be changed up to a complete replacement. RPC interface provided by the Tezos node reflects this architecture:
* There are shell-level endpoints that don't depend on a particular protocol;
* There are block-level endpoints which can response differently depending on the protocol version (block level).

PyTezos wrapper reflects the RPC endpoint scheme and provides so called "chaining" interface for creating requests. For instance if you want to query `GET /a/b/c/d` it would be `a.b.c.d()`. If it's a `POST` request it would be `a.b.c.d.post()`. RPC documentation is integrated to the query engine and you can check available methods on each step, just execute e.g. `a.b.c` in a notebook cell.

## Network
Before moving on to playing with RPC we need to connect to one of Tezos networks. Aside from the `mainnet` which carries value, there are several public [testnets](https://tezos.gitlab.io/introduction/test_networks.html), plus anyone can spawn his own [sandboxnet](https://tezos.gitlab.io/developer/flextesa.html) e.g. for debugging dapps.  

First of all we create a PyTezos client instance and initialize the network (by default it uses the network with latest known protocol activated):

In [1]:
from pytezos import pytezos

pytezos = pytezos.using(shell='mainnet')
pytezos.shell.node.uri

'https://rpc.tzkt.io/mainnet/'

It can also be node uri:

In [2]:
pytezos = pytezos.using(shell='https://mainnet-tezos.giganode.io/')

Note that `using` method returns a new instance of PyTezos client and does not modifies the existing one.

## Chain
There can be multiple chains at a particular moment in a single network:
* There is a special testing period in the voting process during which new protocol is battle tested but detached from the real funds.
* Due to network delays or other reasons two or more block producers can simultaneously extend main chain essentially creating forks. With time (usually 1-2 blocks) all such chains except one with the highest ["fitness"](https://tezos.gitlab.io/whitedoc/proof_of_stake.html?highlight=fitness#fitness) become orphans (not adopted by others) and then obsolete.

Keep in mind, that your RPC provider can be at an orphan state at some point of time and rollback only after several minutes.  

Most time you will work with the `main` chain:

In [3]:
pytezos.shell.chains.main


Properties
.path  # /chains/main
.node  # https://mainnet-tezos.giganode.io/ ()

RPC endpoints
.blocks
.chain_id
.checkpoint
.invalid_blocks
.mempool

Helpers
.watermark()

Note the output — this is the builtin documentation mentioned earlier. In particular list of RPC methods is useful for exploration or if you forget something.

## Block
Blocks can be accessed by hash, level (height), special identifiers (`head`, `genesis`), or offsets (`head-123`, `head+123`).  
In a nutshell, block consists of a header and a list of operations. Block hash (blake2b) is actually derived from the byte representation of its header.

In [4]:
pytezos.shell.chains.main.blocks.head.hash()

'BLP93MhbrKvMKLsBuy8UErpwn1QYVCR7DHwhD3geWfXgemZY3jm'

There are useful shortcuts for most common scenarios, e.g. accessing head.

In [5]:
pytezos.shell.head.header()

{'protocol': 'PsCARTHAGazKbHtnKfLzQg3kms52kSRpgnDY982a9oYsSXRLQEb',
 'chain_id': 'NetXdQprcVkpaWU',
 'hash': 'BLP93MhbrKvMKLsBuy8UErpwn1QYVCR7DHwhD3geWfXgemZY3jm',
 'level': 1131056,
 'proto': 6,
 'predecessor': 'BKkHEv6Qq4cWo14QkfQSNFpWiczCxtZvBGUgmRrD62DEf8YbHwj',
 'timestamp': '2020-09-16T15:51:58Z',
 'validation_pass': 4,
 'operations_hash': 'LLobEPiM5zFMSwFQVMxttzS9sTtBJPhqp5Q7shkhp5ZpxFVk8TGDL',
 'fitness': ['01', '0000000000074230'],
 'context': 'CoV64qaFykBGjNzStJDdTu7NyCPrmPVMPRmFFHVYrdzS1zrGxiZD',
 'priority': 0,
 'proof_of_work_nonce': 'ba45727c3e590600',
 'signature': 'siga8iyjVFyKDkycPSSX47hs4S9byVPEtScXsL5C82yuWD1Xm4qDDKkJvUTjuQUhE5Uwk4NiqiXHfQq53EoRwwrUJpxxQ9vA'}

Block header fields are also divided into "shell" (protocol independent) and protocol-specific.  
Read detailed description of each field in the [whitedoc](https://tezos.gitlab.io/whitedoc/proof_of_stake.html#blocks)  

A useful source of information is the block receipt containing state changes happened in the result of applying this block. There are also operation receipts which we will examine a bit later.

In [6]:
pytezos.shell.blocks[10000].metadata()

{'protocol': 'PtCJ7pwoxe8JasnHY8YonnLYjcVHmhiARPJvqcC6VfHT5s8k8sY',
 'next_protocol': 'PtCJ7pwoxe8JasnHY8YonnLYjcVHmhiARPJvqcC6VfHT5s8k8sY',
 'test_chain_status': {'status': 'not_running'},
 'max_operations_ttl': 60,
 'max_operation_data_length': 16384,
 'max_block_header_length': 238,
 'max_operation_list_length': [{'max_size': 32768, 'max_op': 32},
  {'max_size': 32768},
  {'max_size': 135168, 'max_op': 132},
  {'max_size': 524288}],
 'baker': 'tz3RDC3Jdn4j15J7bBHZd29EUee9gVB1CxD9',
 'level': {'level': 10000,
  'level_position': 9999,
  'cycle': 2,
  'cycle_position': 1807,
  'voting_period': 0,
  'voting_period_position': 9999,
  'expected_commitment': False},
 'voting_period_kind': 'proposal',
 'nonce_hash': None,
 'consumed_gas': '0',
 'deactivated': [],
 'balance_updates': [{'kind': 'contract',
   'contract': 'tz3RDC3Jdn4j15J7bBHZd29EUee9gVB1CxD9',
   'change': '-16000000'},
  {'kind': 'freezer',
   'category': 'deposits',
   'delegate': 'tz3RDC3Jdn4j15J7bBHZd29EUee9gVB1CxD9',
  

In order to better understand what receipt fields mean, check out the articles about [Proof-of-stake](https://tezos.gitlab.io/whitedoc/proof_of_stake.html) in Tezos and how [self-amendmend](https://learn.tqtezos.com/files/self-amendment.html) works.  
There are also several shortcuts for most common cases:

In [7]:
pytezos.shell.head.cycle()

276

In [8]:
pytezos.shell.head.voting_period()

34

Lastly, you can examine some rare and interesting cases, e.g. protocol activation:

In [9]:
protocol_parameters = pytezos.shell.blocks[1].protocol_parameters()  # genesis block
protocol_parameters['commitments'][:5]  # recomended allocations (aka commitments)

[['btz1LKddKfC9cEitY8Q5bMnGS9yj9uPWnmX68', '4974860242'],
 ['btz1LKj1hxiXZUkZGuCMTNtGos8npjo9VjFnJ', '14770866000'],
 ['btz1LKkKTQ34S1BSrcEwYFpVRhm2inDYfjCMg', '1551209817'],
 ['btz1LKmBsCgNKpkWq3hBBqrPPyEmE1XegouD8', '12000000000'],
 ['btz1LKnYAGSyGRJiY7pieJVK9RR6v3e2ko6QS', '9000000000']]

## Block slices, cycles, and voting periods
This not a wrapped RPC endpoint but a convenient extension allowing to manipulate with block slices and iterate over larger timeframes namely [**cycles**](http://doc.tzalpha.net/whitedoc/proof_of_stake.html#cycles) and [**voting periods**](http://doc.tzalpha.net/whitedoc/voting.html#periods).

In [10]:
pytezos.shell.blocks[20:30]()

[['BLGD9HqcQiyKa9eQ8pBuNVPR3bkc3XgaT9BgxLaeE2jdvcpCYEX',
  'BLmJDttDAPHdS7THb1oaRVzhakZoUJTLdxz1sq49Vw6E2vDxBpG',
  'BLUHiyMJSwhVETPprN14kQcQodM5n6yJtTtyD9rTkJbAwD2UCXX',
  'BLrX28Eay45rbaCCMuwSH8q6uPTXcB9LiMHy699NfSssbNLzxzF',
  'BM3YXmkqgKzWevHe28iKDN1JxXa2XV8rjUwpz7PMyrS1y5FS1NQ',
  'BLk8qc7hdBRtB6cBJ2PL2UTtu1eEkaxkpVYTZz113rZEZswYU2k',
  'BLyAEwaXShJuZasvUezHUfLqzZ48V8XrPvXF2wRaH15tmzEpsHT',
  'BMWVEwEYw9m5iaHzqxDfkPzZTV4rhkSouRh3DkVMVGkxZ3EVaNs',
  'BMM2XoWLyzhYfXDc7p5uZ5a4n8318GoXbJiFUKejAQthjANxpm3',
  'BKzHjmWqi7wXqrjbKi5NVZ7UGH8hQ9mrkpaABrzSyEfquULxnan',
  'BKy1nQDvKvic22MzCkqUNeVEENDZzkaoBkVSiK7uBeC8mwWEUNy']]

Block slice has `get_range` method that returns boundary levels which can be useful when working with cycles/voting periods.

In [11]:
pytezos.shell.cycles[-1].get_range()

(1130497, 1131056)

In [12]:
pytezos.shell.voting_periods[1].get_range()

(1, 32768)

The main purpose of block slices (aside from determining boundaries) is operation search, e.g. when you have recently sent an operation and want to find it by hash:

In [13]:
pytezos.shell.blocks[1129600:1129700].find_operation('onw8xuA8SKNdJo4VUf5vyhJKRmggWRVuLAavCbRnmQvE1zr65KF')

{'protocol': 'PsCARTHAGazKbHtnKfLzQg3kms52kSRpgnDY982a9oYsSXRLQEb',
 'chain_id': 'NetXdQprcVkpaWU',
 'hash': 'onw8xuA8SKNdJo4VUf5vyhJKRmggWRVuLAavCbRnmQvE1zr65KF',
 'branch': 'BLCWACrJjKFGEwUecZQjna3eiWCy21xCz1cu3TdpmSiLN1KCmJz',
 'contents': [{'kind': 'transaction',
   'source': 'tz1bNL8YciKPtCuKNzQWxVF8Bnm1h3sd8sbB',
   'fee': '8113',
   'counter': '5373301',
   'gas_limit': '78140',
   'storage_limit': '0',
   'amount': '0',
   'destination': 'KT1VG2WtYdSWz5E7chTeAdDPZNy2MpP8pTfL',
   'parameters': {'entrypoint': 'default',
    'value': {'prim': 'Right',
     'args': [{'prim': 'Left',
       'args': [{'bytes': 'c139928f72887b79db36eb848dbe78aafa429c980421cc7e88a6d117a22949b1'}]}]}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1bNL8YciKPtCuKNzQWxVF8Bnm1h3sd8sbB',
      'change': '-8113'},
     {'kind': 'freezer',
      'category': 'fees',
      'delegate': 'tz1ei4WtWEMEJekSv8qDnu9PExG6Q8HgRGr3',
      'cycle': 275,
      'change': '8113'}],
    'oper

## Operation group
If we take a look at the list of operations included into the block we will find a list of lists actually, moreover the size of the outer list is always 4.

In [14]:
pytezos.shell.head.operation_hashes()

[['oo2BUCNwz6G4L5rb3AhicaN9edJNP646vAyz6f4eHRuuybyxWq9',
  'ooH77gGGD6pMNYtrmWzzz1yxwvqkXzjE94XeYHnvuy3u5SrQkYh',
  'oo7qN3AGx4eTWLEqaPr8P5FJstdG52D8DPuTMm2aiwtzr9B7rue',
  'oo2kw6RSkUSz3weWhv1C5dE688AhfS6quyMgx6K27MdQwSLNobw',
  'op3FbdodNFrPQU1BZiSJnzZyHtx2WpnAEn2Zrzd8Xgm1DiyNM7N',
  'oo7UmkyVr2Scz9gdbpZHipDmuYHytaQR9BgdfjguDuqKxR5zbH9',
  'onwNrn3k7wjAZ4vQCbBqJzrFQh3ozcY1ZbiWqw4EA4nR8aYQ59e',
  'oo1e5NDtFMyyVQMmRMYUKYrMRYdQm4TLQgnUw5R6gPJoupbPcEC',
  'opL52uGpWFtTSezdihuLaEbfS84s1iXY2ryLiJDQEfBsCCeWdxL',
  'opWdVVyvvXsrqLy6zZHA63UCxKNbF9Fe6SKNmJbMPC8RbCwmfJk',
  'onmoRk4W7HLVsroSNsKNyitDLpfC9hf9y1YdXWWmX74zTXEMzHA',
  'onkeQZuNmDGrWN3uVeBtStHeDAc4BeCJUQEWt3NAeBDaABrY8Na',
  'onibycjE8JERgnoSp3JBvoZLY5R8czmbdZoca5RrzQWfM3Mz4yW',
  'ooEoLToW78btzrR2LcDp883orn4DYfMFU7W7ZDxVYY8Ddod9n7Q',
  'opR8rhznAUanCFuFh8G1HNdfXsMSfjHCydmLKSR5e2fdAvzmuzT',
  'oodxFfKfSD4kfpHJarE1nVzdNropHK7rtEoKDif53AopZME4xkX',
  'ooJvBd43SEWv3xksS2Ymnwb4jAvUA6g98yZ2emMLKrurwBCXQFv',
  'oon9Y4ZQEgEKMCs8R3u6kDy2fyEF

These are the so-called validation passes, the operations are grouped in such a way as to detect the error as early as possible.  

1. **Endorsement** — contains only operations with kind "endorsement", which affect the fitness of the block;
2. **Vote** — contains "ballot" and "proposal" operations used in the self-amending mechanism;
3. **Anonymous** — contains operations that do not have signatures thus cannot be linked to any particular account (until some level);
4. **Manager** — contains signed operations that do change account states.

Read more about various [operation kinds](https://medium.com/tqtezos/lifecycle-of-an-operation-in-tezos-248c51038ec2).

In [15]:
pytezos.shell.head.operations


Properties
.path  # /chains/main/blocks/head/operations
.node  # https://mainnet-tezos.giganode.io/ ()

()
All the operations included in the block.
:returns: Array

[] Find operation by hash.

:param item: Operation group hash (base58)
:rtype: OperationQuery

Helpers
.anonymous
.endorsements
.find_ballots()
.find_origination()
.find_upvotes()
.managers
.votes

Note shortcuts for validation passes, also you can search through all operations by hash.

So far we used the term "operation" although were actually speaking about "operation groups". Here's how an operation group is constructed:

```
Operation Group  <-- hash refers to the entire group
└ Content #0
  └ Main operation
  └ Internal operation results  <-- only if main operation is "manager" and applied
    └ Internal operation #0
    ...
    └ Internal operation #m
...
└ Content #n
```

Multicontent structure essentially allows operation batching, however an important rule must be followed: **only operations with the same validation pass can be batched**.  

Internal operations may occur as a result of calls to smart contracts, which may invoke other contracts in their turn. Pay attention to the flat structure (intuitively expected tree-like) of internal operations. In Tezos contracts cannot synchronously invoke other contracts but return a list of spawned operations (i.e. async cross-contract calls) instread. The ordering of contract calls is quite unusual, [BFS](https://forum.tezosagora.org/t/problems-with-concurrency/1771) is used (usually DFS):

```
A      1
└ B    2
  └ C  4
  └ D  5
  E    3
  └ F  6
  └ G  7
```

How to determine the order of a particular operation from the RPC response:
1. Main operations — by **counter**, which is tied to a specific account (signatory) and is incremented by one after each successful operation;
2. Internal operations — by **nonce**, which is being reset on every main operation.

PyTezos provides convenient iterators for operations and operation results (receipts):

In [16]:
from pytezos.operation.result import OperationResult

In [17]:
opg = pytezos.shell \
    .blocks[1113876] \
    .operations['opHN29qMFpK6fMswaWmb2WYis7JggqF6fzJtysaB5LRAkzC17Zk']()
    
for op in OperationResult.iter_contents(opg):
    print(op['kind'], 'internal' if op['internal'] else '')

transaction 
transaction internal
transaction internal


In [18]:
for res in OperationResult.iter_results(opg):
    print(res['status'])

applied
applied
applied


You can also aggregate some stats or extract specific fields:

In [19]:
opg = pytezos.shell \
    .blocks[850141] \
    .operations['opGUCeryD3N4pAjLPZNqWVcgkxjPyTVf4y9uRxDzguWaBQvStgj']()

OperationResult.burned(opg)

257

In [20]:
OperationResult.originated_contracts(opg)

['KT1XHAmdRKugP1Q38CxDmpcRSxq143KpEiYx']

## Context
Context is the blockchain state, it keeps actual information about accounts, balances, contract data, and other. Note, that a node has to run in a special ["archive"](https://tezos.gitlab.io/user/history_modes.html) mode in order to provide access to the context. Also note that many data providers close particularly heavy context endpoints.  

The most common scenario using context data is querying account state (balance, storage, code, baker-specific info):

In [21]:
pytezos.shell.head.context.contracts['tz1cf7vkqZiDwSJRpKN4W3NNHPHAXgomM3GE']()

{'balance': '24969932',
 'delegate': 'tz1NortRftucvAkD1J58L32EhSVrQEWJCEnB',
 'counter': '6761142'}

In [22]:
pytezos.shell.head.context.contracts['KT1VG2WtYdSWz5E7chTeAdDPZNy2MpP8pTfL'].storage()

{'prim': 'Pair', 'args': [{'int': '4'}, {'prim': 'Unit'}]}

There is no public documentation for context endpoints, but you can utilize the one integrated to PyTezos:

In [23]:
pytezos.shell.head.context.raw.json


Properties
.path  # /chains/main/blocks/head/context/raw/json
.node  # https://mainnet-tezos.giganode.io/ ()

()
¯\_(ツ)_/¯
:param depth: ¯\_(ツ)_/¯
:returns: Object

RPC endpoints
.active_delegates_with_rolls
.big_maps
.block_priority
.commitments
.contracts
.cycle
.delegates
.delegates_with_frozen_balance
.ramp_up
.rolls
.votes

For example, you can query all keys of a particular big map:

In [24]:
pytezos.shell.head.context.raw.json.big_maps.index[31].contents()[:5]

['exprvSvW5ffvG4epdyhVxqutvzeWXdCTx8AshBRn2JNxpuTkzyt92o',
 'exprvNUxqffUzU5UzdSFh2jYWpA9YmFtj5thBqCAhKAdEZZRLdD2pv',
 'exprvMg58jmxM2S5y7Cev4H8E4rQgchCmjkFYTTXX4ZUoR59NfEi27',
 'exprvKJfvczXea5o7zn9R7Q3cA1C3zPbR9Y4XjtfGiyFcdcPp6ef6U',
 'exprvJmH6JcTnhLkR6BFioC9qqmT1b1AvcxmjCcekxS84oHwNbonFC']

## Boosting RPC
PyTezos uses several tricks to improve experience with RPC requests. First, it caches deterministic requests (i.e. to past blocks) but only for known public networks. This option can be turned off:

In [25]:
pytezos.shell.node.caching = False

In addition to that, you can use a pool of RPC nodes to make concurrent requests:

In [26]:
from concurrent.futures import ThreadPoolExecutor
import logging

logging.basicConfig(level=logging.DEBUG)

registry = pytezos.using('mainnet-pool') \
    .contract('KT1ChNsEFxwyCbJyWGSL3KdjeXE28AY1Kaog')

bakers_addresses = ['tz1abmz7jiCV2GH2u81LRrGgAFFgvQgiDiaf', 
                    'tz1WvL7MKyCuUHfC3FxPFyiXcy8hHTBT3vjE', 
                    'tz1PWCDnz783NNGGQjEFFsHtrcK5yBW4E2rm']

def big_map_get(address):
    return registry.big_map_get(address, 'head')

with ThreadPoolExecutor(max_workers=3) as executor:
    snapshot = executor.map(big_map_get, bakers_addresses)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rpc.tzkt.io:443
DEBUG:urllib3.connectionpool:https://rpc.tzkt.io:443 "GET /mainnet/chains/main/blocks/head/context/contracts/KT1ChNsEFxwyCbJyWGSL3KdjeXE28AY1Kaog HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): tezos-prod.cryptonomic-infra.tech:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): tezos-prod.cryptonomic-infra.tech:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (3): tezos-prod.cryptonomic-infra.tech:443
DEBUG:urllib3.connectionpool:https://tezos-prod.cryptonomic-infra.tech:443 "GET /chains/main/blocks/head/context/contracts/KT1ChNsEFxwyCbJyWGSL3KdjeXE28AY1Kaog/storage HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rpc.tezrpc.me:443
DEBUG:urllib3.connectionpool:https://tezos-prod.cryptonomic-infra.tech:443 "GET /chains/main/blocks/head/context/contracts/KT1ChNsEFxwyCbJyWGSL3KdjeXE28AY1Kaog/storage HTTP/1.1" 20